- https://catalog.us-east-1.prod.workshops.aws/workshops/59ea75b5-532c-4b57-982e-e58152ae5c46/ko-KR/20-build-agent/21
- https://github.com/netsgo0319/ecomm-review-agent/blob/main/lab01_review_sentiment_analyzer/sentiment_analyzer/agent.py

In [1]:
!pip install -r requirements.txt --quiet --force-reinstall --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.54 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.56.1 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0,

In [2]:
import logging
import orjson

from strands import Agent
from strands.models import BedrockModel
from rich.logging import RichHandler

In [3]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Configure logging with rich (common in modern Python projects)
logging.basicConfig(
    level=logging.INFO,
    datefmt="[%Y-%m-%d %H:%M:%S]",
    handlers=[
        RichHandler(
            rich_tracebacks=True,
            markup=True,
            show_time=True,
            show_level=True,
            show_path=True,
        )
    ],
)

In [4]:
SYSTEM_PROMPT = """
    당신은 리뷰 감정 분석 전문가입니다.
    리뷰 텍스트를 분석하여 감정을 분류하고 점수를 매겨주세요.

    <감정분류>
    - positive: 긍정적 감정 (만족, 기쁨, 추천 등)
    - negative: 부정적 감정 (불만, 실망, 비추천 등)
    - neutral: 중립적 감정 (객관적 서술, 단순 정보 등)
    </감정분류>

    <점수범위>
    감정 점수: -1.0 (매우 부정) ~ 1.0 (매우 긍정)
    </점수범위>

    <주의사항>
    - 반어법, 비꼬는 표현('정말 좋네요' + 부정적 맥락)을 주의깊게 감지해주세요
    - 복합 감정이 있는 경우 전체적인 주된 감정을 파악해주세요
    - 한국어 완곡 표현을 고려해주세요: '나쁘지 않다'는 긍정적, '그럭저럭'은 보통 만족
    - 도메인별 전문용어의 맥락을 파악해주세요 (예: 게임의 '어려움'은 긍정적일 수 있음)
    - 비교 표현('~보다 나아요')의 상대적 의미를 고려해주세요
    - 텍스트가 너무 짧거나 모호한 경우 confidence를 낮게 설정해주세요
    </주의사항>

    <출력형식>
    결과를 JSON 형식으로 반환해주세요. 답변에 백틱이나 코드 블록 포맷(```json, ```python 등)을 붙이지 마세요. :
    {
    "sentiment": "positive|negative|neutral",
    "score": 0.8,
    "confidence": 0.9,
    "reason": "분석 근거"
    }
    </출력형식>
"""

In [5]:
# 입력 예시
review_content = "제품 정말 좋아요! 음질도 훌륭하고 배터리도 오래 갑니다."

In [6]:
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
)

sentiment_agent = Agent(
    model=bedrock_model,
    system_prompt=SYSTEM_PROMPT,
)

# Strands Agent 호출
result = sentiment_agent(review_content)

{
  "sentiment": "positive",
  "score": 0.8,
  "confidence": 0.95,
  "reason": "명확한 긍정 표현('정말 좋아요', '훌륭하고')과 구체적인 장점 언급(음질, 배터리 지속성)으로 강한 만족감을 표현하고 있습니다."
}

실전 Tip 💡
- https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/

Lab 2에서 다룹니다.

In [7]:
result_text = str(result)

In [8]:
type(result_text)

str

In [9]:
print(result_text)

{
  "sentiment": "positive",
  "score": 0.8,
  "confidence": 0.95,
  "reason": "명확한 긍정 표현('정말 좋아요', '훌륭하고')과 구체적인 장점 언급(음질, 배터리 지속성)으로 강한 만족감을 표현하고 있습니다."
}



In [10]:
orjson.loads(result_text)

{'sentiment': 'positive',
 'score': 0.8,
 'confidence': 0.95,
 'reason': "명확한 긍정 표현('정말 좋아요', '훌륭하고')과 구체적인 장점 언급(음질, 배터리 지속성)으로 강한 만족감을 표현하고 있습니다."}